In [137]:
from JogoDeForca import JogoDeForca
import re

jogo = JogoDeForca()
tamanho_palavra = jogo.novo_jogo()
tamanho_palavra

7

In [138]:
jogo.tentar_letra("a")
content = jogo.content
content


['a',
 'Aarao',
 'aba',
 'abacate',
 'abacateiro',
 'abacateiros',
 'abacates',
 'abacaxi',
 'abacaxis',
 'abaciais',
 'abacial',
 'abaco',
 'abacos',
 'abade',
 'abades',
 'abadessa',
 'abadessas',
 'abadia',
 'abadias',
 'abafa',
 'abafada',
 'abafadas',
 'abafadica',
 'abafadicas',
 'abafadico',
 'abafadicos',
 'abafado',
 'abafados',
 'abafai',
 'abafais',
 'abafam',
 'abafamento',
 'abafamentos',
 'abafamo',
 'abafamos',
 'abafando',
 'abafar',
 'abafara',
 'abafaram',
 'abafaramos',
 'abafarao',
 'abafaras',
 'abafardes',
 'abafarei',
 'abafareis',
 'abafarem',
 'abafaremo',
 'abafaremos',
 'abafares',
 'abafaria',
 'abafariam',
 'abafariamos',
 'abafarias',
 'abafarieis',
 'abafarmo',
 'abafarmos',
 'abafas',
 'abafasse',
 'abafasseis',
 'abafassem',
 'abafassemos',
 'abafasses',
 'abafaste',
 'abafastes',
 'abafava',
 'abafavam',
 'abafavamos',
 'abafavas',
 'abafaveis',
 'abafe',
 'abafei',
 'abafeis',
 'abafem',
 'abafemo',
 'abafemos',
 'abafes',
 'abafo',
 'abafou',
 'abaix

In [149]:
class Jogador():
    def __init__(self, tamanho_palavra, content):
        self.dict_acertos = {}
        self.tamanho_palavra = tamanho_palavra
        alfabeto = "abcdefghijklmnopqrstuvwxyz"
        self.dict_probabilidades_letra = {letra : 0 for letra in alfabeto}
        self.regex = "^[a-z]{7}$"
        self.palavras_possiveis = content
        self.letras_tentadas = ""

    def atualiza_regex(self):
        parte_central = self.regex[3:8]
        print(parte_central)
        for letra,indice in self.dict_acertos.items():
            
            self.regex = "\\b"+"\w{indice}"+"\\b"

    def atualiza_base_de_palavras(self):
        p_finais = []
        regex = re.compile(self.regex)
        for palavra in self.palavras_possiveis:
            if regex.search(palavra):
                p_finais.append(palavra)
        # p_possiveis = [palavra for palavra in content if regex.search(palavra) == True]
        return p_finais
    
    def escolhe_melhor_jogada(self):
        melhor_letra = ""
        maior_valor = 0
        for letra in self.dict_probabilidades_letra:
            if self.dict_probabilidades_letra[letra] >= maior_valor and letra not in self.letras_tentadas:
                maior_valor = self.dict_probabilidades_letra[letra]
                melhor_letra = letra
                self.letras_tentadas += letra
        return melhor_letra
    
    def atualiza_dict_acertos(self, letra, lista_acertos):
        if lista_acertos != []:
            self.dict_acertos[letra] = lista_acertos
 
    def atualiza_probabilidades(self):
        for palavra in self.palavras_possiveis:
            letras_ja_adicionadas = ""
            for letra in palavra.lower():
                if letra not in letras_ja_adicionadas:
                    self.dict_probabilidades_letra[letra] += 1
                    letras_ja_adicionadas += letra
            
    def jogada(self, letra, lista_acertos):
        self.atualiza_dict_acertos(letra, lista_acertos)
        self.atualiza_regex()
        self.atualiza_base_de_palavras()
        self.atualiza_probabilidades()
        return self.escolhe_melhor_jogada()

    



In [150]:
jogador = Jogador(tamanho_palavra,content)
print(jogo.palavra)
letra = ""
lista_acertos = []
i = 0
while i < 3:
    letra = jogador.jogada(letra,lista_acertos)
    lista_acertos = jogo.tentar_letra(letra)
    print(letra)
    print(lista_acertos)
    print(jogador.regex)
    print(jogador.dict_probabilidades_letra)

-z]{7
a
None
^[a-z]{7}$
{'a': 205869, 'b': 29765, 'c': 88431, 'd': 72123, 'e': 182017, 'f': 29257, 'g': 33225, 'h': 18343, 'i': 150638, 'j': 8432, 'k': 21, 'l': 63866, 'm': 112612, 'n': 93878, 'o': 140078, 'p': 50276, 'q': 8175, 'r': 165303, 's': 164467, 't': 92633, 'u': 61609, 'v': 41462, 'w': 18, 'x': 7930, 'y': 7, 'z': 18473}
-z]{7
e
None
\b\w{indice}\b
{'a': 411738, 'b': 59530, 'c': 176862, 'd': 144246, 'e': 364034, 'f': 58514, 'g': 66450, 'h': 36686, 'i': 301276, 'j': 16864, 'k': 42, 'l': 127732, 'm': 225224, 'n': 187756, 'o': 280156, 'p': 100552, 'q': 16350, 'r': 330606, 's': 328934, 't': 185266, 'u': 123218, 'v': 82924, 'w': 36, 'x': 15860, 'y': 14, 'z': 36946}
w{ind
r
None
\b\w{indice}\b
{'a': 617607, 'b': 89295, 'c': 265293, 'd': 216369, 'e': 546051, 'f': 87771, 'g': 99675, 'h': 55029, 'i': 451914, 'j': 25296, 'k': 63, 'l': 191598, 'm': 337836, 'n': 281634, 'o': 420234, 'p': 150828, 'q': 24525, 'r': 495909, 's': 493401, 't': 277899, 'u': 184827, 'v': 124386, 'w': 54, 'x': 2379

Traceback (most recent call last):
  File "C:\Users\Gustavin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Gustavin\AppData\Local\Temp\ipykernel_62452\1445536085.py", line 6, in <module>
    letra = jogador.jogada(letra,lista_acertos)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gustavin\AppData\Local\Temp\ipykernel_62452\549897268.py", line 53, in jogada
    self.atualiza_probabilidades()
  File "C:\Users\Gustavin\AppData\Local\Temp\ipykernel_62452\549897268.py", line -1, in atualiza_probabilidades
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Gustavin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-pack